In [13]:
from sklearn.datasets import load_digits
digits = load_digits()

In [14]:
print(digits.data.shape)
print(digits.target[44])

(1797, 64)
7


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

In [16]:
data_tensor = torch.Tensor(digits.data)
target_tensor = F.one_hot(torch.Tensor(digits.target).long(), 10).float()

dataset = TensorDataset(data_tensor, target_tensor)

# Create dataloader
batch_size = 64
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Simple FFN approach

class FFN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        linear_relu_stack = nn.Sequential(
            nn.Linear(8*8, 32),
            nn.ReLU(),
            nn.Linear(32, 10),
            # nn.ReLU(),
            # nn.Linear(512, 10),
        )

    def forward(self, x):
        # x = self.flatten(x)
        logits = linear_relu_stack(x)
        return logits
        return F.softmax(logits, dim=0)

model = FFN()

epochs = 100
batch_size = 10

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for i in range(epochs):
    for j in range(0, len(data_tensor), batch_size):
        pred = model(data_tensor[j:j+batch_size])
        
        target = target_tensor[j:j+batch_size]

        loss = loss_fn(pred, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if i % 10 == 0:
        print(loss.item())

# Double check
print(torch.argmax(model(data_tensor[5])))
print(target_tensor[5])

NEAT PYTORCH

to do
genome species population stuff
crossover
improve parallelization i think

Parallelize the eval of each nn. The nn is not parallelizable because mm cannot be used. 

In [17]:
# NEAT Classes

from collections import deque, defaultdict
import random

torch.manual_seed(42)

class Node:
    def __init__(self, id_, input_=False, output=False):
        self.id = id_
        self.is_input = input_
        self.is_output = output

        self.val = torch.Tensor(0) # Tensors
        self.num_incoming_connections = 0
        
        if not input_ and not output:
            # Output has special handling when init because it starts off fully connected
            self.num_incoming_connections = 1
            
        self.received = 0 # Keep track of nodes received before applying activation function
        
class ConnectionGene:
    def __init__(self, in_node, out_node, weight, innov_num):
        self.in_node = in_node
        self.out_node = out_node
        self.weight = weight # Weights are tensors
        self.innov_num = innov_num

        self.enable = True # If node is disabled, it CAN be reenabled



class NN(nn.Module):
    glob_node_num = 0
    glob_innov_num = 0
    
    def __init__(self, input_dim, output_dim, crossover=False):
        super(NN, self).__init__()

        if crossover:
            # Initialized in a different way
            self.input_dim = input_dim
            self.output_dim = output_dim
            
            self.nodes = []
            
            self.connections = []
            self.num_disabled = 0 # Might not need this anymore     

        
        else:
            # When init a new model, the node nums and innov should be the same as any other new initialized model
            local_node_num = 0
            local_innov_num = 0
    
            self.input_dim = input_dim
            self.output_dim = output_dim
    
            self.nodes = []
            
            self.connections = []
            self.num_disabled = 0 # Might not need this anymore
    
            # Initalize a fully connected NN with no hidden layers
            for _ in range(input_dim):
                new_node = Node(local_node_num, True)
                self.nodes.append(new_node)
                local_node_num += 1
    
            for _ in range(output_dim):
                new_node = Node(local_node_num, False, True)
                self.nodes.append(new_node)
                local_node_num += 1
    
                for i in range(input_dim):
                    conn = ConnectionGene(self.nodes[i], new_node, torch.randn(1), local_innov_num)
                    self.connections.append(conn)
                    local_innov_num += 1
                    new_node.num_incoming_connections += 1 # Each output starts off fully connected to input
    
            NN.glob_node_num = local_node_num
            NN.glob_innov_num = local_innov_num

    def reset(self):
        for i in self.nodes:
            i.val = 0
            i.received = 0

    def clone(self):
        # Create a new NN instance with same input/output dims
        new_nn = NN(self.input_dim, self.output_dim)

        new_nn.num_disabled = self.num_disabled
        
        # Deep copy nodes
        new_nn.nodes = []
        for node in self.nodes:
            new_node = Node(node.id, node.is_input, node.is_output)
            new_node.val = node.val
            new_node.num_incoming_connections = node.num_incoming_connections
            new_node.received = node.received
            new_nn.nodes.append(new_node)

        # Deep copy connections
        new_nn.connections = []
        for conn in self.connections:
            # Find the corresponding new nodes by id
            in_node = next(n for n in new_nn.nodes if n.id == conn.in_node.id)
            out_node = next(n for n in new_nn.nodes if n.id == conn.out_node.id)

            new_conn = ConnectionGene(in_node, out_node, conn.weight, conn.innov_num)
            new_conn.enable = conn.enable
            new_nn.connections.append(new_conn)

        return new_nn
        
    # def forward(self, x):
    #     if len(x) != self.input_dim:
    #         raise ValueError("Input dim is not correct")
    
    #     self.reset()
    
    #     # Set input node values
    #     for idx, val in enumerate(x):
    #         self.nodes[idx].val = val
    
    #     queue = deque()
    #     for node in self.nodes:
    #         if node.received == node.num_incoming_connections:
    #             queue.append(node)
    
    #     while queue:
    #         curr_node = queue.popleft()
    
    #         for conn in self.connections:
    #             if not conn.enable:
    #                 continue
    #             if conn.in_node != curr_node:
    #                 continue
    
    #             out_node = conn.out_node
    #             out_node.val += curr_node.val * conn.weight
    #             out_node.received += 1
    
    #             if out_node.received == out_node.num_incoming_connections:
    #                 if not out_node.is_output:
    #                     out_node.val = max(0, out_node.val)  # ReLU
    #                 queue.append(out_node)
    
    #     logits = torch.tensor([n.val for n in self.nodes if n.is_output])
    #     #     # Cross entropy loss provides softmax so just returning logits
    #     return logits

    def forward(self, x: torch.Tensor):
        if x.shape[1] != self.input_dim:
            raise ValueError("Input dim is not correct")
        
        batch_size = x.shape[0]
        self.reset()
    
        # Create batch versions of node values and received counts
        for node in self.nodes:
            node.vals = torch.zeros(batch_size)
            node.received = torch.zeros(batch_size, dtype=torch.int)
    
        # Set input values
        for idx in range(self.input_dim):
            self.nodes[idx].vals = x[:, idx]
    
        # Start with nodes whose incoming connections are already satisfied
        queue = deque()
        for node in self.nodes:
            if node.num_incoming_connections == 0:
                queue.append(node)
    
        while queue:
            curr_node = queue.popleft()
    
            for conn in self.connections:
                if not conn.enable:
                    continue
                if conn.in_node != curr_node:
                    continue
    
                out_node = conn.out_node
                out_node.vals += curr_node.vals * conn.weight
                out_node.received += 1
    
                # Only enqueue if all inputs are received
                # Note: vectorized check — adds node to queue if all samples are ready
                if (out_node.received == out_node.num_incoming_connections).all():
                    if not out_node.is_output:
                        out_node.vals = torch.relu(out_node.vals)
                    queue.append(out_node)
    
        # Collect logits from output nodes
        output_vals = [node.vals for node in self.nodes if node.is_output]
        logits = torch.stack(output_vals, dim=1)  # shape: (batch_size, num_outputs)
        return logits

    
    def weight_perturbation(self, quiet):
        rand_conn_id = torch.randint(0, len(self.connections), (1,)).item()
        
        mean = 0.0
        std_dev = 0.1
        
        noise = torch.randn_like(self.connections[rand_conn_id].weight) * std_dev + mean
        self.connections[rand_conn_id].weight += noise
        
        if not quiet:
            print(f'connection {rand_conn_id} weight perturbated to {self.connections[rand_conn_id].weight.item():.2f}')
        
    def weight_modification(self, quiet):
        rand_conn_id = torch.randint(0, len(self.connections), (1,)).item()

        self.connections[rand_conn_id].weight = torch.randn(1)
        
        if not quiet:
            print(f'connection {rand_conn_id} weight modified to {self.connections[rand_conn_id].weight.item():.2f}')

    def add_connection(self, quiet):
        max_attempts = 100  # prevent infinite loop
        for _ in range(max_attempts):
            rand_node_in = self.nodes[torch.randint(0, len(self.nodes), (1,)).item()]
        
            if rand_node_in.is_output:
                continue
        
            rand_node_out = self.nodes[torch.randint(0, len(self.nodes), (1,)).item()]
        
            if rand_node_out == rand_node_in or rand_node_out.is_input:
                continue
        
            # Skip if connection already exists
            if any(conn.in_node == rand_node_in and conn.out_node == rand_node_out for conn in self.connections):
                continue
        
            # Optional: skip if this would form a cycle
            # if self.creates_cycle(rand_node_in, rand_node_out):
            #     continue
        
            conn = ConnectionGene(rand_node_in, rand_node_out, torch.randn(1), NN.glob_innov_num)
            self.connections.append(conn)
            NN.glob_innov_num += 1
            rand_node_out.num_incoming_connections += 1
            
            if not quiet:
                print(f"Connection created from node {rand_node_in.id} to node {rand_node_out.id} (innovation #{NN.glob_innov_num})")

            return
            
        if not quiet:
            print("Failed to add connection after max attempts.")

    def add_node(self, quiet):            
        rand_conn_id = torch.randint(0, len(self.connections), (1,)).item()

        while not self.connections[rand_conn_id].enable:
            rand_conn_id = torch.randint(0, len(self.connections), (1,)).item()

        # Splits an existing connection by adding a node
        self.connections[rand_conn_id].enable = False
        self.num_disabled += 1
        
        new_node = Node(NN.glob_node_num)
        self.nodes.append(new_node)
        NN.glob_node_num += 1
        
        conn = ConnectionGene(self.connections[rand_conn_id].in_node, new_node, torch.randn(1), NN.glob_innov_num)
        self.connections.append(conn)
        NN.glob_innov_num += 1
        
        conn = ConnectionGene(new_node, self.connections[rand_conn_id].out_node, torch.randn(1), NN.glob_innov_num)
        self.connections.append(conn)
        NN.glob_innov_num += 1
        
        if not quiet:
            print(f'connection {rand_conn_id} split')

    def toggle_connection(self, quiet):
        rand_conn_id = torch.randint(0, len(self.connections), (1,)).item()

        if self.connections[rand_conn_id].enable:
            self.num_disabled -= 1
        else:
            self.num_disabled += 1
        self.connections[rand_conn_id].enable = not self.connections[rand_conn_id].enable

        if not quiet:
            print(f'connection {rand_conn_id} toggled to {self.connections[rand_conn_id].enable}')

    def mutate(self, quiet=False):
        # For this experiment i use 60 weight perturbation, 60 weight mutation, 5 add connection, 3 add node, 2 toggle
        # Each one is chosen independently of each other
        # Does not include crossover. That cannot be done by itself
        new_model = self.clone()

        # Do mutations on new_model
        if random.random() < 0.60:
            new_model.weight_perturbation(quiet)
        if random.random() < 0.60:
            new_model.weight_modification(quiet)
        if random.random() < 0.05:
            new_model.add_connection(quiet)
        if random.random() < 0.03:
            new_model.add_node(quiet)
        if random.random() < 0.02:
            new_model.toggle_connection(quiet)
    
        return new_model

In [229]:
def crossover(info1, info2):
    # Equal Fitness
    # Might not implement this rn because it doesnt happen that much
    if info1['loss'] == info2['loss']:
        pass

    # Find fitter model
    fit_model, less_fit_model = (info2['model'], info1['model']) if info1['loss'] > info2['loss'] else (info1['model'], info2['model'])

    # New model starts off as clone of more fit
    new_model = NN(fit_model.input_dim, fit_model.output_dim, True)

    fit_conn_pointer = less_fit_conn_pointer = 0

    while fit_conn_pointer != len(fit_model.connections) - 1 and less_fit_conn_pointer != len(less_fit_model.connections) - 1:
        # IDK

    return new_model

IndentationError: expected an indented block after 'for' statement on line 13 (655297904.py, line 14)

In [25]:
# hyperparameters for measuring compatibility from https://nn.cs.utexas.edu/downloads/papers/stanley.cec02.pdf
c1 = 1.0
c2 = 1.0
c3 = 3.0
delta_thresh = 4.0

def measure_compatibility(genome1, genome2):
    genome1_conns = {i.innov_num: i.weight for i in genome1.connections}
    genome2_conns = {i.innov_num: i.weight for i in genome2.connections}

    innovs1 = set(genome1_conns.keys())
    innovs2 = set(genome2_conns.keys())

    max_innov1 = max(innovs1) if innovs1 else 0
    max_innov2 = max(innovs2) if innovs2 else 0
    max_innov = max(max_innov1, max_innov2)

    # Matching genes: innovation numbers in both genomes
    matching = innovs1.intersection(innovs2)
    # Calculate average weight difference for matching genes
    if matching:
        weight_diff = sum(abs(genome1_conns[i] - genome2_conns[i]) for i in matching) / len(matching)
    else:
        weight_diff = 0

    # Excess genes: genes whose innovation number is greater than max innovation number of other genome
    excess = 0
    for innov in innovs1:
        if innov > max_innov2:
            excess += 1
    for innov in innovs2:
        if innov > max_innov1:
            excess += 1

    # Disjoint genes: genes that do not match and are not excess
    disjoint = (len(innovs1 - innovs2) + len(innovs2 - innovs1)) - excess

    # Normalization factor N
    N = max(len(genome1_conns), len(genome2_conns))
    if N < 20:
        N = 1  # as per original NEAT paper for small genomes

    delta = (c1 * excess / N) + (c2 * disjoint / N) + (c3 * weight_diff)
    
    # return delta
    return True if delta < delta_thresh else False

m1 = NN(64, 10)
m2 = NN(64, 10)

measure_compatibility(m1, m2)

True

In [18]:
model_infos = [{"model": NN(64, 10), "loss": float('inf')} for _ in range(10)]
top_models = [info["model"] for info in model_infos[:5]]

for model in top_models:
        model_infos.append({"model": model.mutate(), "loss": float('inf')})


connection 11 weight modified to 1.86
connection 398 weight perturbated to 0.97
connection 615 toggled to False
connection 292 weight perturbated to 0.15
connection 212 weight perturbated to -0.21
connection 136 weight modified to 0.38
connection 121 weight modified to -1.29
15


In [20]:
# "Training loop"

# Hyperparameters
num_models_init = 20
epochs = 100
num_sample_trained_on = 1000
input_dim = 64
output_dim = 10

# Init
model_infos = [{"model": NN(input_dim, output_dim), "loss": float('inf')} for _ in range(num_models_init)]

loss_fn = nn.CrossEntropyLoss()

# Potentially can have mutate percents as hyperparameters here. Lowkey dont want to refactor rn

for i in range(epochs):
    for info in model_infos:
        info["model"].reset()
    
    with torch.no_grad():
        for info in model_infos:
            model = info["model"]
            total_loss = 0.0
            total_samples = 0

            for data_batch, label_batch in loader:
                output = model(data_batch)
                loss = loss_fn(output, label_batch)
                total_loss += loss.item() * data_batch.size(0)
                total_samples += data_batch.size(0)
            
            info["loss"] = total_loss / total_samples

    # Especiation stuff goes here before you rank the models
    
    ranked_models = sorted(model_infos, key=lambda x: x["loss"])
    print(f"top model loss: {ranked_models[0]['loss']:.2f}")

    top_models = [info for info in ranked_models[:int(num_models_init/2)]]


    # for model in top_models:
    #     model_infos.append({"model": model["model"].mutate(), "loss": float('inf')})

    # Test removing old models for faster performance and only mutating top
    
    new_models = [{"model": ranked_models[0]['model'].mutate(True), "loss": float('inf')} for _ in range(len(top_models))]
    # new_models = [{"model": model["model"].mutate(True), "loss": float('inf')} for model in top_models]
    model_infos = top_models + new_models

        

# And then mutate the top models

top model loss: 63.74
top model loss: 62.59
top model loss: 62.54


KeyboardInterrupt: 

In [101]:
model = NN(64, 10)

In [96]:
model(data_tensor[0])

tensor([ -8.2188, -15.4980, -83.0724,  33.3792, -99.2694,   3.3089,  24.0452,
        -37.0231, -81.9885,  47.4248])

In [102]:
model.mutate()

connection 597 weight perturbated to 1.89


NN()

In [ ]:
data_tensor[0]

In [ ]:
model(data_tensor[0])